In [ ]:
from cs336_basics.bpe import *

# 1. 训练 BPE
vocab, merges = bpe(
    input_path="/mnt/e/bpe_data/lfs-data/owt_valid.txt",
    vocab_size=10000,
    special_tokens=["<|endoftext|>", "<|endoftext|><|endoftext|>"],
)

print("longest token: ", max(vocab.items(), key = lambda x: len(x[1])))
# 2. 保存词汇表（vocab）
with open("/home/smingtao01/Download/CS336/llm-from-scratch-assignment1-basics/cs336_basics/owt_vocab.txt", 'w', encoding="utf-8") as f1:
    for token_id, token_bytes in vocab.items():
        # 将字节解码为字符串，处理解码错误
        try:
            token_str = token_bytes.decode('utf-8')
        except UnicodeDecodeError:
            # 如果无法解码为 UTF-8，使用转义表示
            token_str = repr(token_bytes)[2:-1]  # 去掉 b'...' 的 b'' 部分
        
        # 写入格式：ID<TAB>Token
        f1.write(f"{token_id}\t{token_str}\n")

print(f"词汇表已保存: {len(vocab)} 个词元")

# 3. 保存合并规则（merges）
with open("/home/smingtao01/Download/CS336/llm-from-scratch-assignment1-basics/cs336_basics/owt_merges.txt", "w", encoding='utf-8') as f2:
    # 检查 merges 的数据结构
    # merges 可能是 dict 或 list，需要根据实际情况处理
    
    if isinstance(merges, dict):
        # 如果是字典：{(b'a', b'b'): b'ab', ...}
        for (byte1, byte2), merged_bytes in merges.items():
            try:
                pair_str = f"{byte1.decode('utf-8')} + {byte2.decode('utf-8')}"
                merged_str = merged_bytes.decode('utf-8')
            except UnicodeDecodeError:
                pair_str = f"{repr(byte1)} + {repr(byte2)}"
                merged_str = repr(merged_bytes)
            
            f2.write(f"{pair_str} -> {merged_str}\n")
    
    elif isinstance(merges, list):
        # 如果是列表：[(b'a', b'b'), (b'b', b'c'), ...]
        for i, (byte1, byte2) in enumerate(merges, 1):
            try:
                pair_str = f"{byte1.decode('utf-8')} {byte2.decode('utf-8')}"
            except UnicodeDecodeError:
                pair_str = f"{repr(byte1)} {repr(byte2)}"
            
            f2.write(f"{i}: {pair_str}\n")

print(f"合并规则已保存: {len(merges)} 个规则")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/e/bpe_data/lfs-data/owt-valid.txt'